In [3]:
from skyfield.api import Topos, load

stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')
#for sat in satellites:
#    print(sat)

Loaded 70 satellites


In [ ]:
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['ISS (ZARYA)']
print(satellite)

In [ ]:
t_now = ts.now()
dt_now = ts.now().utc_datetime()
dt_morning = dt_now.replace(hour=0,minute=0,second=0, microsecond=0)
dt_evening = dt_now.replace(hour=23,minute=59,second=59, microsecond=0)
print (dir(dt_morning))
#two_hours = ts.utc(2014, 1, 20, 0, range(0, 120, 20))

print(dt_morning.strftime('%Y-%m-%d %H:%M'))
print(dt_evening.strftime('%Y-%m-%d %H:%M'))

In [ ]:
from skyfield.api import EarthSatellite

ts = load.timescale(builtin=True)
line1 = '1 25544U 98067A   14020.93268519  .00009878  00000-0  18200-3 0  5082'
line2 = '2 25544  51.6498 109.4756 0003572  55.9686 274.8005 15.49815350868473'
satellite_tle = EarthSatellite(line1, line2, 'ISS (ZARYA)', ts)
print(satellite_tle)
print(satellite_tle.epoch.utc_jpl())

In [ ]:
t = ts.utc(2014, 1, 23, 11, 18, 7)

days = t - satellite.epoch
print('{:.3f} days away from epoch'.format(days))

if abs(days) > 14:
    satellites = load.tle_file(stations_url, reload=True)

In [ ]:
bluffton = Topos('40.8939 N', '83.8917 W')
t0 = ts.utc(2014, 1, 23)
t1 = ts.utc(2014, 1, 24)

tm = ts.utc(dt_morning)
te = ts.utc(dt_evening)
t, events = satellite.find_events(bluffton, tm, te, altitude_degrees=30.0)
for ti, event in zip(t, events):
    name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

In [ ]:
# You can instead use ts.now() for the current time
# t = ts.utc(2014, 1, 23, 11, 18, 7)
t = ts.now()
geocentric = satellite.at(t)
print(geocentric.position.km)

In [ ]:
subpoint = geocentric.subpoint()
print('Latitude:', subpoint.latitude)
print('Longitude:', subpoint.longitude)
print('Elevation (m):', int(subpoint.elevation.m))

In [ ]:
difference = satellite - bluffton
print(difference)

In [ ]:
topocentric = difference.at(t)
print(topocentric.position.km)

alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The ISS is above the horizon')

print(alt)
print(az)
print(int(distance.km), 'km')

In [ ]:
ra, dec, distance = topocentric.radec()  # ICRF ("J2000")

print(ra)
print(dec)

In [ ]:
ra, dec, distance = topocentric.radec(epoch='date')

print(ra)
print(dec)

In [ ]:
eph = load('de421.bsp')
satellite = by_name['ISS (ZARYA)']

two_hours = ts.utc(2014, 1, 20, 0, range(0, 120, 20))
sunlit = satellite.at(two_hours).is_sunlit(eph)
print(sunlit)

In [ ]:
for ti, sunlit_i in zip(two_hours, sunlit):
    print('{}  {} is in {}'.format(
        ti.utc_strftime('%Y-%m-%d %H:%M'),
        satellite.name,
        'sunlight' if sunlit_i else 'shadow',
    ))